<li class="dropdown-items active">
<label title="是 Yes" class="btn-block">
    <span class="pull-left">
    <i class="maticon"></i></span> 
    <span>是 Yes</span>
</label></li>


In [1]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [28]:
url = 'https://xmuxg.xmu.edu.cn/app/214'

driver = webdriver.Chrome('/Users/Alexey/PycharmProjects/Scrapping/chromedriver')
driver.get(url)
# check whether logged in
# wait to load up TODO: change to a different method


In [29]:
loaded_url = driver.current_url
if 'login' in loaded_url:
    # log in here then
    print('waiting to click')
    loging_button = driver.find_element_by_xpath("//button[contains(.,'统一身份认证')]")

    loging_button.click()

    

waiting to click


In [30]:
    login_field = driver.find_element_by_name("username")
    login_field.click()
    print('clicking login')
    # login_field.clear()
    login_field.send_keys('')
    
    password_field = driver.find_element_by_name("password")
    password_field.click()
    print('clicking password')
        # password_field.clear()
    password_field.send_keys('')

clicking login
clicking password


In [31]:
    # press login/登录
    loging_button = driver.find_element_by_xpath("//button[contains(.,'登录/Login')]")
    loging_button.click()


print('logged in')

logged in


In [32]:
# CHECK DATE
from datetime import datetime

curdate = datetime.today().strftime('%Y-%m-%d')

print(curdate)

2020-03-29


In [33]:
if len(driver.find_elements_by_xpath("//span[text()[contains(.,'" + curdate + "')]]")) > 0:
    print("date is correct:", curdate)
else:
    print("there's something wrong with date "+ curdate +", but we'll continue anyway")

date is correct: 2020-03-29


In [12]:
# CHECK TIME
hours = int(datetime.today().strftime('%H'))
minutes = int(datetime.today().strftime('%M'))

if hours >= 16 and minutes >= 30:
    print("too late for the daily health report")
    driver.close()
    exit()
else:
    print("time's alright:", hours, ':', minutes)


time's alright: 0 : 15


In [13]:
# Finally getting to the reporting part,
menu_button = driver.find_element_by_xpath("//div[contains(@class, 'tab')][2]")
# menu_button = driver.find_element_by_xpath("//div[text()='我的菜单')]")
menu_button.click()


In [15]:
# Finally getting to the reporting part,
menu_button = driver.find_element_by_xpath("//div[contains(@class, 'tab')][2]")
# menu_button = driver.find_element_by_xpath("//div[text()='我的菜单')]")
menu_button.click()


In [25]:
confirmation = driver.find_elements_by_xpath("//div[contains(@class, 'form-control dropdown-toggle')]")

In [26]:
if len(driver.find_elements_by_xpath("//span[text()[contains(.,'是 Yes')]]/ancestor::label[@class='btn-block']")) > 0:
    print("confirmation is already yes, consider checking website yourself")
    # clicking outside
    # driver.find_element_by_xpath("//body").click()
else:
    # hoping that the last element is the confirmation one
    confirmation[-1].click()
    yes_button = driver.find_element_by_xpath("//span[text()[contains(.,'是 Yes')]]/ancestor::label[@class='btn-block']")
    yes_button.click()

In [27]:
confirmation

[<selenium.webdriver.remote.webelement.WebElement (session="152b5f547079e6ede4ff6cc7976ad527", element="5a471cb9-149d-488a-ab99-5929a0df04eb")>,
 <selenium.webdriver.remote.webelement.WebElement (session="152b5f547079e6ede4ff6cc7976ad527", element="13bc1ede-c2ac-4df0-b3d2-f833381d7d76")>,
 <selenium.webdriver.remote.webelement.WebElement (session="152b5f547079e6ede4ff6cc7976ad527", element="519a9cc8-a237-4bd2-980e-5afede4e2240")>,
 <selenium.webdriver.remote.webelement.WebElement (session="152b5f547079e6ede4ff6cc7976ad527", element="eb48f9c9-cc06-443e-b41e-c3a0d7ec13ee")>,
 <selenium.webdriver.remote.webelement.WebElement (session="152b5f547079e6ede4ff6cc7976ad527", element="d020e53f-46b6-48ad-935c-82a623714918")>,
 <selenium.webdriver.remote.webelement.WebElement (session="152b5f547079e6ede4ff6cc7976ad527", element="17954d5c-5526-489f-8a64-c5ce46adfe3e")>,
 <selenium.webdriver.remote.webelement.WebElement (session="152b5f547079e6ede4ff6cc7976ad527", element="eea4f1c6-cabf-4677-9310-77

In [22]:
save_button = driver.find_elements_by_xpath("//span[text()[contains(.,'保存')]]")

save_button[-1].click()

In [23]:
driver.switch_to.alert.accept()

In [ ]:
driver.close()



